In [3]:
import numpy as np

In [2]:
from data_cleaning import clean_data
data = clean_data()

FileNotFoundError: [Errno 2] No such file or directory: 'Vol3ProjectData.csv'

In [3]:
df, users_who_star = data

In [50]:
def encode(df, expanded=True):
    column_labels = df['concept_id'].unique()
    row_labels = df['user_id'].unique()
    #Initialize Matrix
    # Layer 0 is a T/F (0/1) indicator if the word has been learned at all
    # Layer 1 is a T/F (0/1) indicator if the word has been starred or mistaken
    df = df.sort_values(by=['user_id','words_studied'])
    unique = list(set(df['user_id']))
    unique = np.array(unique)
#     print(unique)
#     print(df['user_id'].values[0])
#     print(np.where(unique == df['user_id'].values[0])[0][0])
    #if we want to expand the matrix (connor needed the data from the unexpanded matrix sometimes)
    if expanded:
        #create an empty 3 dimensional matrix full of zeros
#         data = np.zeros((len(row_labels),len(column_labels),len(column_labels)))
        data = csr_matrix((len(df)-len(row_labels),len(column_labels)), dtype=np.int8)
        labels = np.zeros(len(df)-len(row_labels))

        n = 0
        #cycle through every user and add ones to the words theyve learned in order 
        #(data[i][0] will have a 1 at the first word user i learned after 1 iteration, data[i][1] will have a 1 at the first and second word user i learned after 2 iterations)
        for i in range(len(row_labels)):
            for j in range(list(df['user_id'].values).count(df['user_id'].values[n])-1):
                if j > 0:
                    data[i,:] = data[i-1,:] 
                data[i,np.where(column_labels == df.values[n][1])[0][0]] = 1
                labels[i] = np.where(column_labels == df.values[n+1][1])[0][0]
                n += 1
        return data, labels
    #if not expanded then just do all the ones at once and create 2 matrices (one for encounters of words and one for starred/mistaken words)                
    else:
        data = np.zeros((len(row_labels),len(column_labels),2))
        for index, row in df.iterrows():
            i = np.where(row_labels == row['user_id'])[0][0]
            j = np.where(column_labels == row['concept_id'])[0][0]
            #Add indices and data to lists to construct matrix
            data[i,j,0] = 1
            data[i,j,1] = max((row['starred'], row['mistaken']))
        return data

In [51]:
import pandas as pd
from scipy.sparse import csr_matrix
df = pd.read_csv("Vol3StarredData.csv",index_col=0)
data = encode(df)

/home/connor/anaconda3/lib/python3.8/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
/home/connor/anaconda3/lib/python3.8/site-packages/scipy/sparse/_index.py:116: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


KeyboardInterrupt: 